# Conteo de píxeles en las máscaras

El objetivo de este notebook es calcular en un archivo **JSON** cuantos pixeles se poseen por cada clase en cada una de las máscaras para emplear estos datos preprocesados en otros notebooks. Para ello, ejecutamos un proceso de carga del raster de cada máscara y contamos cuantos pixeles hay por cada clase, totalizamos y almacenamos el resultado en un archivo.

## Librerias

Carga de las librerias que vamos a emplear

In [1]:
from sklearn.preprocessing import MinMaxScaler
from typing import List
from matplotlib import colors
from skimage import exposure
from tqdm.notebook import tqdm

import rasterio as rio
import rasterio.plot as rio_plot
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import json

# Semilla pseudoaleatoria
np.random.seed(24)

# Tamaño de las figuras
plt.rcParams["figure.figsize"] = (20,10)

# Ruta base de la carpeta de datos
DATA_BASE = "/home/ggonzr_cloud/deeplearn/data"

### Carga de imágenes

A continuación vamos a definir dos variables de acceso a la ruta de las imagenes satelitales en el sistema de archivos al igual que para las máscaras.

In [2]:
# Directorios de las etiquetas y la imagenes
LABEL_DIR = f"{DATA_BASE}/ref_landcovernet_v1_labels"
IMAGE_DIR = f"{DATA_BASE}/ref_landcovernet_v1_source"

In [3]:
# Listar los subdirectorios de cada carpeta: Etiquetas
label_dirs = glob.glob(f"{LABEL_DIR}/*")
print(f"Número de etiquetas disponibles: {len(label_dirs)}")

Número de etiquetas disponibles: 1982


In [4]:
# Listar los subdirectorios de cada carpeta: Etiquetas
image_dirs = glob.glob(f"{IMAGE_DIR}/*")
print(f"Número de etiquetas disponibles: {len(image_dirs)}")

Número de etiquetas disponibles: 82430


En las siguientes dos celdas podrá observar un ejemplo de la ruta de acceso tanto de las imagenes satelitales como de sus etiquetas. Cada una de ellas se representa en una carpeta

In [5]:
label_dirs[0]

'/home/ggonzr_cloud/deeplearn/data/ref_landcovernet_v1_labels/ref_landcovernet_v1_labels_36PZC_07'

In [6]:
image_dirs[0]

'/home/ggonzr_cloud/deeplearn/data/ref_landcovernet_v1_source/ref_landcovernet_v1_source_33LYE_11_20180818'

# Conteo de pixeles por las etiquetas

En la siguiente celta definimos una función auxiliar para crear un cáscaron de la respuesta que vamos a almacenar. Esta función retorna un valor por defecto cuyos valores serán cambiados cuando haya calculado la respuesta

In [7]:
def create_response() -> dict:
    labels_tag = list(range(1,8))
    zeros_list = np.zeros(len(labels_tag))
    zeros_list = zeros_list.tolist()
    cont_dict = dict(zip(labels_tag, zeros_list))
    cont_dict["chip_id"] = ""
    cont_dict["label_folder"] = ""
    return cont_dict

En la siguiente función recibimos la ruta de carga de la máscara para cargarla, contar los pixeles por cada clase y retornar una repuesta con el número de pixeles por cada clase, el ID de la máscara y la ruta de esta máscara en el sistema de archivos

In [8]:
def pixel_count(path_label: str) -> dict:
    folder_name = path_label.split("/")[-1]
    name_split = folder_name.split("_")
    chip_name = f"{name_split[-2]}_{name_split[-1]}"
    
    label_shape = glob.glob(f"{path_label}/label*")[0]    
    
    # Cargar la mascara
    label_mask = None
    with rio.open(label_shape, 'r') as rf:
        label_mask = rf.read(1)
    
    # Realizar el conteo
    class_tag, pix_count = np.unique(label_mask, return_counts=True)
    class_tag = class_tag.tolist()
    pix_count = pix_count.tolist()
    
    # Instanciar diccionario de retorno
    rsp = create_response()
    
    # Retorno
    pix_dict = dict(zip(class_tag, pix_count))
    pix_dict["chip_id"] = chip_name
    pix_dict["label_folder"] = path_label
    
    return {**rsp, **pix_dict}

In [9]:
# Procesamiento
process_result = []

In [10]:
for lf in tqdm(label_dirs):
    try:
        pxd = pixel_count(lf)
        process_result.append(pxd)    
    except:
        continue

  0%|          | 0/1982 [00:00<?, ?it/s]

Cuando el proceso termine, almacenamos el resultado en el archivo **JSON** denominado **pixel_report.json**

In [11]:
# Guardar datos
with open(f"{DATA_BASE}/pixel_report.json", "w", encoding="utf-8") as v_f:
    json.dump(process_result, v_f, ensure_ascii=False, indent=4)